In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import os
from datetime import datetime
from MCForecastTools import MCSimulation
import requests
import pandas_datareader as pdr
%matplotlib inline

In [2]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [ ]:
# Set Empty Dictionary
stock_dict = {}

# Set Empty List
stock_list = []

# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")
validate(date)

# While Loop to Pick Stocks
while (True):
    name = input("Add a stock ticker to the list: ").upper()
    validateTick(name)
    stock_dict[name] = date
    stock_list.append(name)
    cont = input("Press 'y' to Add Another Stock, press 'Enter' to exit")
    if cont == "":
        break;

Pick a start date for your analysis (YYYY-MM-DD):  2015-01-01
Add a stock ticker to the list:  TSLA
Press 'y' to Add Another Stock, press 'Enter' to exit MSFT
Add a stock ticker to the list:  y
Press 'y' to Add Another Stock, press 'Enter' to exit y
Add a stock ticker to the list:  MSFT


In [ ]:
def stockCreation(stock_dict, stock_list):
    func_list = []
    count = 0
    for key, value in stock_dict.items():
        globals()[key] = pdr.DataReader(f'{key}','yahoo',value)
        globals()[key].columns = [[key, key, key, key, key, key,],["high", "low", "open", "close", "volume", "adj_close"]]
        func_list.append(globals()[key])
    
    portfolio = func_list[0]
    
    if len(func_list) == 2:
        portfolio = portfolio.merge(func_list[1], left_index=True, right_index=True, suffixes=[f'_{stock_list[0]}', f'_{stock_list[1]}'])
    elif len(func_list) > 2:
        portfolio = portfolio.merge(func_list[1], left_index=True, right_index=True)
        for x in range(len(func_list)-2):
            portfolio = portfolio.merge(func_list[x + 2], left_index=True, right_index=True)

    portfolio_close = []       
    for x in range(len(stock_list)):
        portfolio_close.append(portfolio[stock_list[x]][f"adj_close"])
    
    portfolio_close = pd.DataFrame(portfolio_close)
    
    portfolio_close = portfolio_close.T
    
    portfolio_close.columns = stock_list
        
        
    
    return portfolio, portfolio_close

In [ ]:
portfolio = stockCreation(stock_dict, stock_list)[0]
portfolio

In [ ]:
portfolio_close = stockCreation(stock_dict, stock_list)[1]
portfolio_close

Sharpe Ratios

Definition: average return earned beyond the risk-free rate per unit of volatility, the measure of the price fluctuations of an asset. The greater the value of the Sharpe ratio, the more attractive the risk-adjusted return.

Purpose: The ratio describes the compensation an investor will receive for the extra volatility endured when holding a riskier asset.

In [ ]:
# Calculate annualized Sharpe Ratios
sharpe_ratios = (portfolio.mean()*252) / (portfolio.std()*np.sqrt(252))

# Show results
sharpe_ratios

In [ ]:
# Initialize new figure
plt.figure()

# Visualize Sharpe Ratios as a bar plot
sharpe_ratios.plot(kind="bar", figsize = (20,10), title="Sharpe Ratios");

In [ ]:
# Create a correlation analysis
custom_corr = portfolio.corr()

# Show results
custom_corr